In [ ]:
import pandas as pd
import re
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
a=warnings.filterwarnings('ignore')
import pickle
from sklearn.svm import LinearSVC

tring Cleansing

In [ ]:
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+https\S+", '',text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','',text)
    text = re.sub(r'[^\w\s]','',text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

emmatization

In [ ]:
def lemmatizing_frame(data):
    lempred = WordNetLemmatizer()
    l = len(data)
    for i in range(0,l,1):
        txt = []
        list = data[i].split(" ")
    
    
        for w in list:
            a = lempred.lemmatize(w)
            txt.append(a)
            textprocessed = ' '.join(map(str,txt))
        data[i] = textprocessed

olarity

In [ ]:
def polarity(text):
    return TextBlob(text).sentiment.polarity

entiment Labelling

In [ ]:
def sentiment(label):
    if label <0:
        return "Negative"
    elif label ==0:
        return "Neutral"
    elif label>0:
        return "Positive"

xporting processed csv to train the model<br>
ataset Import

In [ ]:
def ProcessedData(Dataset):
    df = pd.read_csv(Dataset)

    #Dropping columns
    text_df = df.drop(['id', 'user_name', 'user_location', 'user_description', 'user_created',
           'user_followers', 'user_friends', 'user_favourites', 'user_verified',
           'date', 'hashtags', 'source', 'retweets', 'favorites',
           'is_retweet'], axis=1)
    text_df.text = text_df['text'].apply(data_processing)
    lemmatizing_frame(text_df['text'])
    text_df['polarity'] = text_df['text'].apply(polarity)
    text_df['sentiment'] = text_df['polarity'].apply(sentiment)
    text_df = text_df.drop_duplicates('text')
    text_df = text_df.drop(['polarity'],axis=1)
    return text_df.to_csv(r"C:\Users\krupe\OneDrive\Desktop\Sentiment\.csv")

In [ ]:
ProcessedData(r"C:\Users\krupe\OneDrive\Desktop\Sentiment\vaccination_all_tweets.csv")

In [ ]:
df = pd.read_csv(r"C:\Users\krupe\OneDrive\Desktop\Sentiment\.csv")
vectoriser = TfidfVectorizer(ngram_range=(1,2)).fit(df['text'])
vector = "vectoriser.sav"
pickle.dump(vectoriser, open(vector,'wb'))
X = df['text']
Y = df['sentiment']
X = vectoriser.transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
SVCmodel = LinearSVC()
SVCmodel.fit(x_train, y_train)

In [ ]:
filesvc = 'finalized_model_SVC.sav'
pickle.dump(SVCmodel, open(filesvc, 'wb'))